# データの読み込み

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install tensorflow tensorflow-gpu  matplotlib
!pip install tensorflow-io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 35.8 MB/s eta 0:00:00


In [ ]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf 
import tensorflow_io as tfio

In [ ]:
def return_slices(FILEPATH):
    wav = load_mp3_16k_mono(FILEPATH)
    audio_slices = tf.keras.utils.timeseries_dataset_from_array(wav, wav, sequence_length=11200, sequence_stride=11200, batch_size=1)
    return audio_slices

In [ ]:
#森田

person = 2 
# name = ['liu', 'morita']
name = ["liu_HPSS", "morita_HPSS"]

SS = [[] for i in range(person)]

for i in range(person):
  SS[i] = os.path.join('/content/drive/MyDrive/audio_processing/intern/mydata' + '/' + name[i])


In [ ]:
def load_wav_16k_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels) 
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    # Goes from 44100Hz to 16000hz - amplitude of the audio signal
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [ ]:
#森田
import os

ss = [[] for i in range(person)] #大事！！！！！


for n in range(person):

  for i in os.listdir(SS[n]):

    ss[n].append(os.path.join(SS[n],i))
  
  ss[n] = tf.data.Dataset.from_tensor_slices(ss[n])

In [ ]:
#森田
# data = tf.data.Dataset()
sis = []
for i in range(person):

  indices = i
  depth = person    
  print(type(ss))
  sis.append( tf.data.Dataset.zip((ss[i], tf.data.Dataset.from_tensor_slices(([tf.one_hot(indices, depth)]*len(ss[i]))))) )

for i in range(len(sis)-1):
  sis[0] = sis[0].concatenate(sis[i+1])
data = sis[0]

print(len(data))

print(data.as_numpy_iterator().next())

<class 'list'>
<class 'list'>
8
(b'/content/drive/MyDrive/audio_processing/intern/mydata/liu_HPSS/liu_1_footstep_audio_HPSS.wav', array([1., 0.], dtype=float32))


In [ ]:
def load_wav_16k_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels) 
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    # Goes from 44100Hz to 16000hz - amplitude of the audio signal
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

def preprocess(file_path, label):
    wav = load_wav_16k_mono(file_path)
    length_clip_clip = int(0.75 * 16_000)
    spectrogram_list = []
    # print(int(len(wav) * 0.7)+ 160_000*num, int(len(wav) * 0.7)+ 160_000+160_000*num)
    for i in range(int(len(wav)/length_clip_clip)):
      wav_clip_clip = wav[length_clip_clip * i: length_clip_clip + length_clip_clip * i]
      spectrogram = tf.signal.stft(wav_clip_clip, frame_length=320, frame_step=160)
      spectrogram = tf.abs(spectrogram)
      spectrogram = tf.expand_dims(spectrogram, axis=2)
      spectrogram_list.append(spectrogram.numpy())
    return spectrogram_list, label



In [ ]:

import numpy as np
data_numpy = []
for i in data:
  data_numpy.append([i[0].numpy(), i[1].numpy()])

data_numpy_processed_X = []
data_numpy_processed_y = []

for i in data_numpy:
  spectrogram, label = preprocess(i[0],i[1])
  data_numpy_processed_X.append(spectrogram)
  data_numpy_processed_y.append(label)

  # spectrogram
print(len(data_numpy_processed_X))

8


In [ ]:
data_numpy_processed_X = np.array(data_numpy_processed_X)
print(data_numpy_processed_X.shape)

(8,)


<ipython-input-16-94c228e32f46>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data_numpy_processed_X = np.array(data_numpy_processed_X)


In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
train_X, train_y, test_X, test_y = [], [], [], []
for index in range(len(data_numpy_processed_X)):
  data = data_numpy_processed_X[index]
  length_train_data = int(0.7 * len(data))
  train_X_clip = np.array(data[:length_train_data])
  test_X_clip = np.array(data[length_train_data:])
  # train_X_clip = train_X_clip.reshape(train_X_clip.shape[0]*train_X_clip.shape[1], train_X_clip.shape[2],1)
  for i in range(train_X_clip.shape[0]):
    train_X.append(train_X_clip[i])
    train_y.append(data_numpy_processed_y[index])

  for j in range(test_X_clip.shape[0]):
    test_X.append(test_X_clip[j])
    test_y.append(data_numpy_processed_y[index])

train_X = np.array(train_X)
train_y = np.array(train_y)
test_X = np.array(test_X)
test_y = np.array(test_y)



num_majority_decision = int(10/0.75)
num_test_data = int(test_X.shape[0]/num_majority_decision)
test_X, test_y = test_X[:num_test_data*num_majority_decision], test_y[:num_test_data*num_majority_decision]
test_X = test_X.reshape(num_test_data, num_majority_decision, test_X.shape[1], test_X.shape[2],1)
test_y = test_y.reshape(num_test_data, num_majority_decision, test_y.shape[1])


# スケーリング
# train_X = train_X / np.max(train_X)
# test_X = test_X / np.max(test_X)

print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)
print()

(1191, 74, 257, 1)
(1191, 2)
(39, 13, 74, 257, 1)
(39, 13, 2)



In [ ]:

# shuffle
print(train_X.shape)
indices = np.arange(train_X.shape[0])
np.random.shuffle(indices)
train_X = train_X[indices]
train_y = train_y[indices]

# shuffle
print(test_X.shape)
print(test_y.shape)
indices = np.arange(test_X.shape[0])
np.random.shuffle(indices)
test_X = test_X[indices]
test_y = test_y[indices]

(1191, 74, 257, 1)
(39, 13, 74, 257, 1)
(39, 13, 2)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout

# 比較対象として、転移学習なしのモデル

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(train_X.shape[1], 257,1)))
model.add(Conv2D(16, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(32, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile('Adam', loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 72, 255, 16)       160       
                                                                 
 conv2d_1 (Conv2D)           (None, 70, 253, 16)       2320      
                                                                 
 flatten (Flatten)           (None, 283360)            0         
                                                                 
 dropout (Dropout)           (None, 283360)            0         
                                                                 
 dense_2 (Dense)             (None, 32)                9067552   
                                                                 
 dense_3 (Dense)             (None, 64)                2112      
                                                                 
 dense_4 (Dense)             (None, 2)                

##Fit Model, View Loss 

In [ ]:
hist = model.fit(train_X, train_y, epochs=5, validation_split=0.2, batch_size = 32)   
# Calling `save('my_model')` creates a SavedModel folder `my_model`.
# model.save("my_model")
# !zip -r /content/my_model.zip /content/my_model

Epoch 1/5
30/30 [==============================] - 39s 1s/step - loss: 0.6924 - accuracy: 0.6092 - val_loss: 0.6367 - val_accuracy: 0.6569
Epoch 2/5
30/30 [==============================] - 38s 1s/step - loss: 0.5742 - accuracy: 0.6817 - val_loss: 0.4989 - val_accuracy: 0.7155
Epoch 3/5
30/30 [==============================] - 37s 1s/step - loss: 0.3844 - accuracy: 0.8109 - val_loss: 0.3378 - val_accuracy: 0.8494
Epoch 4/5
30/30 [==============================] - 36s 1s/step - loss: 0.2032 - accuracy: 0.9191 - val_loss: 0.2824 - val_accuracy: 0.9079
Epoch 5/5
30/30 [==============================] - 35s 1s/step - loss: 0.1643 - accuracy: 0.9338 - val_loss: 0.3819 - val_accuracy: 0.8577


# 転移学習のモデルの構築

In [ ]:
from tensorflow import keras
import os
# It can be used to reconstruct the model identically.
type_folder = "step_160_data_1_cleansed"
if os.path.exists("/content/drive/MyDrive/audio_processing/intern/tem_var/"+type_folder+"/my_model"):
  base_model = keras.models.load_model("/content/drive/MyDrive/audio_processing/intern/tem_var/"+type_folder+"/my_model")
  print("Yes")

Yes


In [ ]:
for layer in base_model.layers:
  layer.trainable = False
model2 = keras.Sequential()
for layer in base_model.layers[:-1]:
  model2.add(layer)
model2.add(keras.layers.Dense(64, activation="tanh"))
model2.add(keras.layers.Dense(2, activation="softmax"))
model2.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 72, 255, 16)       160       
                                                                 
 conv2d_9 (Conv2D)           (None, 70, 253, 16)       2320      
                                                                 
 flatten_4 (Flatten)         (None, 283360)            0         
                                                                 
 dropout_4 (Dropout)         (None, 283360)            0         
                                                                 
 dense_8 (Dense)             (None, 32)                9067552   
                                                                 
 dense_5 (Dense)             (None, 64)                2112      
                                                                 
 dense_6 (Dense)             (None, 2)                

In [ ]:
model2.compile('Adam', loss='categorical_crossentropy', metrics=["accuracy"])

hist = model2.fit(train_X, train_y, epochs=5, validation_split=0.2, batch_size = 32)   


Epoch 1/5
30/30 [==============================] - 14s 427ms/step - loss: 0.6543 - accuracy: 0.6292 - val_loss: 0.5184 - val_accuracy: 0.7782
Epoch 2/5
30/30 [==============================] - 14s 465ms/step - loss: 0.5280 - accuracy: 0.7532 - val_loss: 0.4627 - val_accuracy: 0.7866
Epoch 3/5
30/30 [==============================] - 13s 445ms/step - loss: 0.4885 - accuracy: 0.7910 - val_loss: 0.4376 - val_accuracy: 0.8243
Epoch 4/5
30/30 [==============================] - 12s 409ms/step - loss: 0.4651 - accuracy: 0.8046 - val_loss: 0.4177 - val_accuracy: 0.8452
Epoch 5/5
30/30 [==============================] - 13s 427ms/step - loss: 0.4496 - accuracy: 0.8120 - val_loss: 0.4005 - val_accuracy: 0.8452


In [ ]:
hist.history.keys()
# plt.plot

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

# **テスト**

In [ ]:
# 転移学習なし、騒音除去ありのモデルの精度を計算
sum_acc = 0
print(test_X.shape)
print(type(test_X[1][1]))
for index in range(len(test_X)):
  res_pos = model.predict(test_X[index])
  result_num = np.argmax(res_pos.sum(axis=0))
  if result_num == np.argmax(test_y[index]):
    sum_acc += 1
print("accuracy", sum_acc/len(test_y))

(39, 13, 74, 257, 1)
<class 'numpy.ndarray'>
1/1 [==============================] - 0s 161ms/step
accuracy 0.9487179487179487


In [ ]:
# 転移学習あり、騒音除去ありのモデルの精度を計算
sum_acc = 0
print(test_X.shape)
print(type(test_X[1][1]))
for index in range(len(test_X)):
  res_pos = model2.predict(test_X[index])
  result_num = np.argmax(res_pos.sum(axis=0))
  if result_num == np.argmax(test_y[index]):
    sum_acc += 1
print("accuracy", sum_acc/len(test_y))

(39, 13, 74, 257, 1)
<class 'numpy.ndarray'>
1/1 [==============================] - 0s 91ms/step
accuracy 1.0


In [ ]:
sum = 0
false_sum = 0
for i in range(len(test_X)):
  score = model.evaluate(test_X[i], test_y[i], verbose = 0)
  res_pos = model.predict(test_X[i], verbose=0)
  result_num = np.argmax(res_pos.sum(axis=0))
  if result_num != np.argmax(test_y[i]):
    false_sum += 1
  print(score[1],result_num == np.argmax(test_y[i])) 
  sum += score[1]
print()
print(sum/len(test_X))
print(false_sum)

0.4615384638309479 False
0.8461538553237915 True
1.0 True
1.0 True
1.0 True
0.5384615659713745 True
0.8461538553237915 True
0.7692307829856873 True
0.9230769276618958 True
0.8461538553237915 True
0.9230769276618958 True
0.4615384638309479 True
0.8461538553237915 True
1.0 True
0.692307710647583 True
0.9230769276618958 True
0.8461538553237915 True
0.9230769276618958 True
0.9230769276618958 True
0.9230769276618958 True
0.9230769276618958 True
0.9230769276618958 True
0.8461538553237915 True
0.9230769276618958 True
1.0 True
0.8461538553237915 True
1.0 True
0.8461538553237915 True
0.9230769276618958 True
0.9230769276618958 True
1.0 True
0.8461538553237915 True
0.3076923191547394 False
0.7692307829856873 True
0.9230769276618958 True
0.692307710647583 True
0.8461538553237915 True
0.9230769276618958 True
0.9230769276618958 True

0.848126239501513
2


In [ ]:
sum = 0
false_sum = 0
for i in range(len(test_X)):
  score = model2.evaluate(test_X[i], test_y[i], verbose = 0)
  res_pos = model2.predict(test_X[i], verbose=0)
  result_num = np.argmax(res_pos.sum(axis=0))
  if result_num != np.argmax(test_y[i]):
    false_sum += 1
  print(score[1],result_num == np.argmax(test_y[i])) 
  sum += score[1]
print()
print(sum/len(test_X))
print(false_sum)

0.8461538553237915 True
1.0 True
0.9230769276618958 True
0.8461538553237915 True
1.0 True
0.692307710647583 True
0.9230769276618958 True
0.7692307829856873 True
0.9230769276618958 True
0.7692307829856873 True
0.9230769276618958 True
1.0 True
0.692307710647583 True
1.0 True
0.8461538553237915 True
1.0 True
0.9230769276618958 True
0.692307710647583 True
0.8461538553237915 True
0.9230769276618958 True
1.0 True
0.692307710647583 True
0.8461538553237915 True
0.9230769276618958 True
0.7692307829856873 True
0.7692307829856873 True
0.9230769276618958 True
0.8461538553237915 True
1.0 True
0.692307710647583 True
1.0 True
0.8461538553237915 True
0.9230769276618958 True
0.692307710647583 True
0.692307710647583 True
0.7692307829856873 True
0.692307710647583 True
0.8461538553237915 True
0.692307710647583 True

0.8500986282642071
0
